In [12]:
# Solución al error de inserción en Supabase

# 1. Importar las bibliotecas necesarias
import pandas as pd
from supabase import create_client
import json
import variablesid

# 2. Configuración de Supabase
supabase_url = variablesid.supabase_url
supabase_key = variablesid.supabase_key
supabase_table = "aonijie"

# 3. Crear cliente de Supabase
supabase = create_client(supabase_url, supabase_key)

# 4. Función para preparar los datos correctamente
def preparar_registro(registro):
    """
    Prepara un registro para insertarlo en Supabase
    - Convierte listas a formato JSON
    - Maneja tipos de datos específicos
    """
    registro_limpio = {}
    
    for clave, valor in registro.items():
        # Convertir listas a formato JSON string
        if isinstance(valor, list):
            registro_limpio[clave] = json.dumps(valor)
        # Convertir NaN a None
        elif pd.isna(valor):
            registro_limpio[clave] = None
        else:
            registro_limpio[clave] = valor
    
    return registro_limpio

# 5. Preparar un solo registro para probar
# Es mejor probar con un solo registro primero para ver si funciona
df = pd.DataFrame({
    'app_sale_price': [10.99],
    'original_price': [20.99],
    'product_detail_url': ['https://ejemplo.com/producto1'],
    'second_level_category_name': ['Electrónicos'],
    'target_sale_price': [9.99],
    'second_level_category_id': [101],
    'discount': [10],
    'product_main_image_url': ['https://ejemplo.com/img1.jpg'],
    'first_level_category_id': [1],
    'target_sale_price_currency': ['USD'],
    'target_app_sale_price_currency': ['USD'],
    'tax_rate': [0.16],
    'original_price_currency': ['USD'],
    'shop_url': ['https://ejemplo.com/tienda1'],
    'target_original_price_currency': ['USD'],
    'product_id': ['P001'],
    'target_original_price': [19.99],
    'product_video_url': ['https://ejemplo.com/video1'],
    'first_level_category_name': ['Tecnología'],
    'promotion_link': ['https://ejemplo.com/promo1'],
    'sku_id': ['SKU001'],
    'evaluate_rate': [4.5],
    'sale_price': [11.99],
    'product_title': ['Producto 1'],
    'hot_product_commission_rate': [5],
    'shop_id': ['S001'],
    'app_sale_price_currency': ['USD'],
    'sale_price_currency': ['USD'],
    'lastest_volume': [100],
    'target_app_sale_price': [9.99],
    'commission_rate': [3],
    'fecha_consulta': ['2025-03-01'],
    'small_images': [['img1.jpg', 'img2.jpg']]
})

# 6. Convertir a diccionario y limpiar los datos
registros_dict = df.to_dict(orient='records')
registros_limpios = [preparar_registro(registro) for registro in registros_dict]

# 7. Intentar con un solo registro primero
try:
    # Primero intenta con un único registro para ver si funciona
    registro_prueba = registros_limpios[0]
    print("Intentando insertar un registro de prueba...")
    print(f"Registro de prueba: {json.dumps(registro_prueba, indent=2)}")
    
    response = supabase.table(supabase_table).insert([registro_prueba]).execute()
    print(f"Éxito! Registro insertado: {len(response.data)} registros")
    
    # Si funciona con uno, intenta con todos
    # Nota: Descomentar esto solo si el registro individual funciona
    # print("Ahora intentando con todos los registros...")
    # response_todos = supabase.table(supabase_table).insert(registros_limpios).execute()
    # print(f"Todos los registros insertados: {len(response_todos.data)} registros")
    
except Exception as e:
    print(f"Error detallado: {str(e)}")
    
    # Si hay error, verificamos la existencia de la tabla
    try:
        print("\nVerificando si la tabla existe...")
        response_check = supabase.table(supabase_table).select("count(*)", count="exact").execute()
        print(f"La tabla existe y tiene {response_check.count} registros")
    except Exception as e2:
        print(f"Error al verificar la tabla: {str(e2)}")
        print("Posible problema: La tabla no existe o no tienes permisos")
    
    # Comprobar permisos
    print("\nVerificando permisos de la API key...")
    try:
        response_permisos = supabase.table(supabase_table).select("*").limit(1).execute()
        print("Los permisos de lectura están OK")
    except Exception as e3:
        print(f"Error al verificar permisos: {str(e3)}")

# 8. Solución alternativa: Usar RPC si está disponible
print("\nIntentando método alternativo con RPC...")
try:
    # Supabase también permite usar funciones RPC definidas en el backend
    # Si tienes una función de inserción en el backend, puedes usarla así:
    # response_rpc = supabase.rpc('insertar_datos', {'datos': registro_prueba}).execute()
    # print(f"RPC ejecutado con éxito: {response_rpc.data}")
    
    # O bien, podemos usar PostgreSQL directamente con esta API:
    # Nota: Esta solución requiere permisos especiales
    print("Este método requiere configuración adicional en Supabase. Descomentar si tienes los permisos adecuados.")
except Exception as e:
    print(f"Error con método alternativo: {str(e)}")

Intentando insertar un registro de prueba...
Registro de prueba: {
  "app_sale_price": 10.99,
  "original_price": 20.99,
  "product_detail_url": "https://ejemplo.com/producto1",
  "second_level_category_name": "Electr\u00f3nicos",
  "target_sale_price": 9.99,
  "second_level_category_id": 101,
  "discount": 10,
  "product_main_image_url": "https://ejemplo.com/img1.jpg",
  "first_level_category_id": 1,
  "target_sale_price_currency": "USD",
  "target_app_sale_price_currency": "USD",
  "tax_rate": 0.16,
  "original_price_currency": "USD",
  "shop_url": "https://ejemplo.com/tienda1",
  "target_original_price_currency": "USD",
  "product_id": "P001",
  "target_original_price": 19.99,
  "product_video_url": "https://ejemplo.com/video1",
  "first_level_category_name": "Tecnolog\u00eda",
  "promotion_link": "https://ejemplo.com/promo1",
  "sku_id": "SKU001",
  "evaluate_rate": 4.5,
  "sale_price": 11.99,
  "product_title": "Producto 1",
  "hot_product_commission_rate": 5,
  "shop_id": "S001",

In [17]:
from supabase import create_client

url = variablesid.supabase_url
key = variablesid.supabase_key
supabase = create_client(url, key)

data = {"columna": "valor1"}
response = supabase.table('tabla').insert(data).execute()
